# Pourquoi développer un modèle de recommandation de métiers par les compétences ?

Diagoriente veut aider les gens à s'orienter dans des voies professionnelles qui leurs correspondent et les aide à trouver du travail. Un des aspects de cette tâche est de les aider à découvrir des métiers qui correspondent à leurs compétences, leurs niveaux d'autonomie et de responsabilité, leurs secteurs d'activités et leurs souhaits explicites. Nous disponsons de données, une ontologie en cours d'amélioration, qui associe des métiers à des compétences. Pour proposer des recommandations de métiers à partir de ces données via l'application Diagoriente, nous avons besoin d'un modèle mathématique et informatique qui nous permette de calculer pour chaque métier un score qui reflète son accessibilité pour une personne en prenant en compte tous ces critères. Comme on ne peut attendre de tous les utilisateurs qu'ils fassent l'inventaire de leurs compétences, le modèle doit les déduire de leurs expériences passées.

Ce document définit les objectifs du modèle, le décrit et illustre son utilisation avec un exemple.


# Quel est l'objectif du modèle ?

Le modèle sera un succès s'il aide des personnes en recherche professionnelle à découvrir des métiers auxquelles elles n'avaient pas pensé avant et qu'elles se sentent capables d'exercer éventuellement via une acquisition supplémentaire de compétence à la mesure de ce qui leur semble possible. L'adéquation entre les métiers proposés et la réalité du marché du travail ne fait pas partie des critères d'évaluation du modèle mais pourra y être ajoutée ultérieurement. On mesurera le succès du modèle en collectant les réponses d'utilisateurs à un questionnaire qui leur demandera d'évaluer si elles ont découvert :

- des métiers auxquels elles n'avaient pas pensés ou avaient écartés,
- parmi les précédents, des métiers qu'elles se sentent capables d'exercer,
- des compétences qu'elles se sentent capables d'acquérir qui les aideraient à accéder à de nouveaux métiers.

Dans le cas de réponses négatives, pour pouvoir mieux comprendre l'insatisfaction vis-à-vis des recommandations et améliorer le modèle, le questionnaire demandera aussi d'évaluer l'adéquation entre les métiers présentés et :

- les compétences de la personne,
- son niveau d'autonomie et de responsabilité,
- son secteur d'activité,
- ses souhaits d'orientation,

et l'adéquation entre les compétences recommandées et les compétences actuelles de la personne.


# Méthode

In [147]:
import pandas as pa
from datetime import datetime

## Description des données

Une ontologie de métiers et compétence disponible et en cours d'amélioration décrit les association entre les métiers, secteurs d'activités et compétences. Elle est disponible sous forme de graphe dans une base de données Neo4J.

In [148]:
import oplc_etl.pipelines.neo4j as etl
data = etl.get_data()

On en extrait les éléments suivants :

- un ensemble de secteurs d'activités

In [149]:
data.sectors

,title,ROME
sector,,
0,"Agriculture et Pêche, Espaces naturels et Espa...",A
6,Arts et Façonnage d'ouvrages d'art,B
15,"Banque, Assurance, Immobilier",C
21,"Commerce, Vente et Grande distribution",D
27,"Communication, Média et Multimédia",E
32,"Construction, Bâtiment et Travaux publics",F
40,"Hôtellerie-Restauration, Tourisme, Loisirs et ...",G
49,Industrie,H
68,Installation et Maintenance,I


- un ensemble de métiers avec leur nom, code ROME, nivea CEC et secteur d'activité,

In [150]:
data.jobs

,title,ROME,level,sector
job,,,,
124,Conduite d'engins agricoles et forestiers,A1101,3,0
125,Bûcheronnage et élagage,A1201,3,0
126,Entretien des espaces naturels,A1202,3,0
127,Aménagement et entretien des espaces verts,A1203,3,0
128,Protection du patrimoine naturel,A1204,3,0
...,...,...,...,...
651,Conduite sur rails,N4301,3,112
652,Contrôle des transports en commun,N4302,3,112
653,Circulation du réseau ferré,N4401,4,112


- un ensemble de compétences

In [151]:
data.skills

,title
skill,
656,"Aménager, entretenir un espace naturel, une forêt"
657,"Sélectionner, régler un outil, une machine, un..."
658,Manipuler un engin de manutention ou de chantier
659,"Paramétrer un logiciel, un outil, un système n..."
660,Utiliser et régler des machines ou équipements...
...,...
1094,"Gérer la relation avec les fournisseurs, sous-..."
1095,Transporter des passagers
1096,Conduire un aéronef


- pour chaque métier, les compétences nécessaires.

In [152]:
data.jobs_skills

skill,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,...,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,2947
job,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
124,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
125,1,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
126,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
127,1,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
128,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
652,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
653,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Les niveaux CEC vont de 1 à 8 mais les métiers présents sont associés à des niveaux de 3 à 7.

In [153]:
data.jobs.level.value_counts()

3    265
4    134
5     85
6     32
7      6
Name: level, dtype: int64

## Description du modèle

Pour proposer à une personne des métiers qui lui correspondent, le modèle doit en premier lieu nous permettre de calculer à la fois une valeur d'accessibilité d'un métier et d'une compétence pour une personne, pour tous les métiers et compétences présents dans les données. Pour proposer des compétences à développer pour accéder à d'avantage de métiers, il doit aussi nous permettre d'estimer le potentiel d'une compétence, c'est-à-dire l'effet d'acquérir une nouvelle compétence ou de développer une compétence actuelle sur l'accessibilité des métiers. Ces valeurs doivent refléter l'adéquation entre les métiers ou compétences et les compétences de la personne, son niveau d'autonomie et de responsabilité (niveau CEC), son secteur d'activité de la personne. Le modèle doit lui permettre d'avoir la main sur tous ces éléments. 

> Résumé des notations
>- $i$: indice dénotant une compétences
>- $l$: indice dénotant un métiers
>- $k$: indice dénotant une expérience
>- $w_k$: poids de l'expérience $k$
>- $\vec j^l$: vecteur de compétences du métier $l$
>- $\vec u$: vecteur de compétences de l'utilisateur
>- $\vec s^i$: vecteur de cooccurrence pour la compétence $i$
>- $c$: niveau CEC de l'utilisateur
>- $c^l$: niveau CEC du métier $l$
>- $M$: nombre de compétences
>- $N$: nombre de métiers
>- $a^l$: accessibilité du métier $l$
>- $a^i$: accessibilité de la compétence $i$
>- $p^i$: potentiel de la compétence $i$

### Expériences

Les compétences de la personne proviennent de ses expériences passées. Elles correspondent aux compétences associées aux métiers exercés. Chaque expérience a une durée et une récence propre qui pondèrent son importance dans le parcours professionnel de la personne. On calcule l'importance importance $w_k$ de l'experience $k$ en …

L'importance d'une expérience croit avec sa durée et décroit avec le temps qui s'est écoulé depuis.

Le secteur d'activité de l'utilisateur est celui de son expérience la plus importante.


### Compétences

Pour un métier $l$, un vecteur $\vec j^l = (j^l_1, \dots, j^l_M)$ (où $M$ est le nombre de compétence) représente les compétences associées : $j^l_i$ vaut 1 si le métier est associé à la compétence $i$, 0 sinon.

Les compétences de l'utilisateur sont obtenues en pondérant l'ensemble des vecteurs métiers de ses expériences $\vec j^{l_1}, \dots, \vec j^{l_K}$ par leur poids $\vec w_k$ et en les sommant, ce qui donne le vecteur 

$$\vec u = \sum_k w_k \vec j_k $$

et nous permet de pondérer les compétences de la personne en fonction de la durée et de la récence de ses expériences.


### Accessibilité d'un métier

L'adéquation entre les compétences d'un utilisateur et celles requises pour un métier $l$ est calculée par le cosinus de l'angle entre les deux vecteurs $\vec u$ et $\vec j_l$ que l'on calcule : 
...

Cette valeur varie entre 1 et 0. Elle vaut 1 quand les deux vecteurs sont collinéaires, c'est-a-dire que l'utilisateur a développé de manière égale toutes les compétences associées au métier et aucune autre. Au contraire, elle vaut 0 quand les deux vecteurs sont orthogonaux, c'est-à-dire que l'utilisateur n'a développé aucune compétence associée au métier.

L'accessibilité d'un métier dépend aussi de l'écart entre le niveau CEC de la personne $c$ et celui du métier $c^l$. Un métier qui a un niveau supérieur à la personne est moins accessible. Comme il y a 8 niveaux, on diminue d'un huitième l'accessibilité d'un métier par niveau au dessus de celui de la personne. Un métier qui a un niveau inférieur n'est pas impacté.

L'accessibilité d'un métier $l$ vaut donc:
$$a^l = \frac{\vec u \cdot \vec j^l}{\lvert\vec u\rvert\lvert\vec j^l\rvert} \times \frac{8 - \max(0,c^l - c)}{8}$$

On donne aussi l'option à l'utilisateur d'obtenir d'abord les métiers du même secteurs d'activité mais cela n'impacte pas le score d'accessibilité. Cette option résulte en un simple tri des métiers qui place en priorité ceux du même secteur que l'utilisateur.


### Accessibilité d'une compétence

D'une manière similaire à l'accessibilité d'un métier, l'accessibilité d'une compétence traduit la facilité pour l'utilisateur à l'acquérir étant donné ses compétences actuelles. 

Nous faisons l'hypothèse qu'une compétence $s$ sera plus facile à acquérir si l'utilisateur a développé des compétences qui sont exercées conjointement à $s$ dans d'autres métiers. Par exemple, la compétence "Observer, visualiser, s'orienter" est exercée conjointement à "Aménager, entretenir un espace naturel" dans 4 métiers: Sylviculteur / Sylvicultrice, Aménagement et entretien des espaces verts, Bûcheronnage et élagage et Entretien des espaces naturels.

Cette hypothèse nous amène à représenter une compétence $s$ par les compétences auxquelles elle est associée par l'intermédiaire des métiers. Soit le vecteur de coocurrence pour la compétence $i$ $\vec s^i = (s^i_1, \dots, s^i_M)$ où $s^i_{i'}$ représente le nombre de métiers dans lesquels la compétence $i$ apparaît conjointement à la compétence $i'$ pour $i \neq i'$. On fixe $s^i_i = 0$ car on ne souhaite pas que le nombre de cooccurrences d'une compétence avec elle-même ait un effet dans les calculs. Il est généralement plus grande que celui de deux compétences différentes et tend à éliminer dans les calculs ci-dessous les relations entre compétences que l'on cherche à faire apparaître.

Nous définissons alors l'accessibilité d'une compétence $i$ par le cosinus de l'angle entre le vecteur de compétences de l'utilisateur $u$ et $s^i$ :
$$a^i = \frac{\vec u \cdot \vec s^i}{\lvert\vec u\rvert\lvert\vec s^i\rvert}$$

Comme pour l'accessibilité d'un métier, l'accessibilité d'une compétence vaut 1 lorsque les deux vecteurs sont colinéaires et 0 lorsqu'ils sont orthogonaux. Elle vaut 1 lorsque l'utilisateur a développé ses compétences de manière proportionnelle au nombre de cooccurrence entre la compétence considérée et les autres. Elle vaut 0 quand l'utilisateur n'a aucune compétence liée à la compétence ciblée via les métiers.


### Potentiel d'une compétence

Nous parlons du potentiel d'une compétence pour évoquer sa capacité à lui donner accès à d'avantage de métiers s'il la développe. On souhaite mettre la priorité sur les métiers les plus accessibles. Augmenter l'accessibilité de métiers peu accessibles semble moins pertinents car ils demanderont à l'utilisateur de grands efforts de mise à niveau.

Pour commencer, l'accessibilité moyenne sur tous les métiers vaut :
$$
\frac{1}{N}
\sum_{l} \frac{\vec u \cdot \vec j^l}{\lvert\vec u\rvert\lvert\vec j^l\rvert}
\times
\frac{8 - \max(0,c^l - c)}{8}
$$

Le potentiel d'une compétence $i$ à augmenter l'accessibilité moyenne des métiers correspond à la dérivée partielle de la quantité précédente par rapport à $u_i$, le poids de la compétence $i$ pour l'utilisateur :
$$
\frac{\partial}{\partial u_i} 
\frac{1}{N} 
\sum_l \frac{\vec u \cdot \vec j^l}{\lvert\vec u\rvert\lvert\vec j^l\rvert}  
\times 
\frac{8 - \max(0,c^l - c)}{8}
$$

où $u_i$ est un élément de $\vec u$. Cela donne :

\begin{align}
& \frac{1}{N} 
  \sum_l \frac{8 - \max(0,c^l - c)}{8} 
  \frac{\partial}{\partial u_i} 
  \frac{\vec u \cdot \vec j^l}{\lvert\vec u\rvert\lvert\vec j^l\rvert}
\\
= & \frac{1}{N} 
    \sum_l \frac{8 - \max(0,c^l - c)}{8}  
    \frac{1}{\lvert\vec j^l\rvert}
    \left(
        \frac{j^l_i}{\lvert\vec u\rvert}
        -
        \frac{u_i \sum_{i'} \vec j^l_{i'}u_{i'}}{\lvert\vec u\rvert^3}
    \right)
\end{align}

Pour privilégier l'augmentation de l'accessibilité des métiers déjà les plus accessibles, on pondère chaque élément de la somme par l'accessibilité du métier. Le potentiel de la compétence $i$ vaut ainsi : 

$$
p^i = \frac{1}{N} 
    \sum_l \frac{8 - \max(0,c^l - c)}{8}  
    \frac{1}{\lvert\vec j^l\rvert}
    \left(
        \frac{j^l_i}{\lvert\vec u\rvert}
        -
        \frac{u_i \sum_{i'} \vec j^l_{i'}u_{i'}}{\lvert\vec u\rvert^3}
    \right)
    a^l
$$




# Un exemple

In [154]:
import oplc_model.model_skill_cooc as m

jobs = m.mk_jobs(data.jobs)
skills = m.mk_skills(data.skills)
sectors = m.mk_sectors(data.sectors)
jobs_skills = m.mk_jobs_skills(data.jobs_skills)

## Expériences

Un utilisateur qui a les expériences suivantes:

In [155]:
exp = m.mk_individual_experiences(
    job_id= [124,125,126],
    begin=[datetime.fromisoformat("2018-06-01"),
              datetime.fromisoformat("2020-01-01"),
              datetime.fromisoformat("2022-01-01"),
             ],
    end=[datetime.fromisoformat("2020-12-31"),
            datetime.fromisoformat("2021-12-31"),
            datetime.fromisoformat("2022-05-31"),
           ],
    )
exp = exp.assign(job_title= [data.jobs.title.loc[124], data.jobs.title.loc[125], data.jobs.title.loc[126]])
exp

,job_id,begin,end,job_title
0,124,2018-06-01,2020-12-31,Conduite d'engins agricoles et forestiers
1,125,2020-01-01,2021-12-31,Bûcheronnage et élagage
2,126,2022-01-01,2022-05-31,Entretien des espaces naturels


obtient un poid plus grand pour sa seconde expérience plus courte mais plus récente que sa première. 

In [156]:
user_model = m.model(
    indiv_exp = exp,
    jobs = jobs,
    jobs_skills = jobs_skills)
 
user_model.experiences

,job_id,begin,end,duration,recency,weight
0,124,2018-06-01,2020-12-31,2.586301,1.494536,0.816728
1,125,2020-01-01,2021-12-31,2.000000,0.498630,1.068152
2,126,2022-01-01,2022-05-31,0.413699,0.084932,0.381313


## Compétences

Son vecteur de compétences est donné par la colonne "weight" ci-dessous :

In [157]:
user_model.skills.assign(Nom_de_la_compétence=skills.loc[user_model.skills.index, "title"])

,weight,Nom_de_la_compétence
skill,,
656,2.266194,"Aménager, entretenir un espace naturel, une forêt"
657,0.816728,"Sélectionner, régler un outil, une machine, un..."
658,0.816728,Manipuler un engin de manutention ou de chantier
659,0.816728,"Paramétrer un logiciel, un outil, un système n..."
660,0.816728,Utiliser et régler des machines ou équipements...
...,...,...
1094,0.000000,"Gérer la relation avec les fournisseurs, sous-..."
1095,0.000000,Transporter des passagers
1096,0.000000,Conduire un aéronef


## Accessibilité des métiers

Les dix métiers les plus accessibles de son secteur d'activité, hors métiers déjà exercés, sont :

In [159]:
ja = m.job_accessibility(
        model = user_model,
        jobs = jobs,
        jobs_skills = jobs_skills,
        same_sector_first = True,
        hide_practiced_jobs = True,
        weigh_by_level_diff = True,
        show_level_min = 1,
        show_level_max = 8,
        )

pa.concat([ja.job_accessibility, jobs], axis="columns").head(10)

,0,title,ROME,level,sector
job,,,,,
127,0.408439,Aménagement et entretien des espaces verts,A1203,3,0
146,0.321598,Horticulture et maraîchage,A1414,3,0
137,0.179556,Arboriculture et viticulture,A1405,3,0
148,0.163911,"Polyculture, élevage",A1416,3,0
128,0.159331,Protection du patrimoine naturel,A1204,3,0
136,0.115903,Aquaculture,A1404,3,0
134,0.115903,Aide agricole de production légumière ou végétale,A1402,3,0
135,0.115903,Aide d'élevage agricole et aquacole,A1403,3,0
145,0.089778,Fermentation de boissons alcoolisées,A1413,3,0


## Accessibilité des compétences

Pour le métier "Aménagement et entretien des espaces verts", les compétences manquantes et leurs accessibilités respectives sont :

In [1]:
skill_cooc = m.skill_cooccurrence(jobs_skills)
sa = m.skill_accessibility(user_model, skill_cooc)

job = 127
missing_skills = ja.skill_gap.loc[127, :].sort_values(ascending=False).loc[lambda x: x >= 1.0].index
missing_skills_accessibility = sa.skill_accessibility.loc[missing_skills].sort_values(ascending=False)
pa.concat([missing_skills_accessibility, skills], axis="columns", join="inner")

NameError: name 'm' is not defined

## Potentiel des compétences

Les compétences à développer pour accéder à d'avantage de métiers sont :

In [161]:
sp = m.skill_potential(
    model=user_model, 
    jobs_skills=jobs_skills, 
    job_accessibility=ja, 
    weigh_by_level_diff=True, 
    weigh_by_job_accessibility=True,
)
pa.concat([sp.average.head(10), skills], axis="columns", join="inner")

,0,title
skill,,
904,0.000993,Utiliser un équipement de protection individuelle
664,0.000769,Mettre en oeuvre les processus et les modes op...
738,0.000659,"Fabriquer, façonner des produits"
657,0.000629,"Sélectionner, régler un outil, une machine, un..."
717,0.000581,"Relever, contrôler, ajuster des mesures et dos..."
692,0.000529,Analyser et contrôler la qualité et la conform...
785,0.000506,"Evaluer, prévenir, et gérer les risques et la ..."
713,0.000454,Approvisionner une machine en matière première...
818,0.000433,"Respecter les règles d'hygiène, de sécurité et..."


Développer la compétence "Utiliser un équipement de protection individuelle" augmenterait l'accessibilité des métiers suivants :

In [162]:
pa.concat(
    [sp.per_job.loc[904, :].sort_values(ascending=False).loc[lambda x: x > 0],
     jobs,
     ],
     axis="columns",
     join="inner")

,904,title,ROME,level,sector
job,,,,,
440,0.094790,Intervention en grande hauteur,I1501,3,68
272,0.072994,Conduite de grue,F1301,3,32
441,0.046490,Intervention en milieu subaquatique,I1502,3,68
273,0.043042,Conduite d'engins de terrassement et de carrière,F1302,3,32
442,0.043042,Intervention en milieux et produits nocifs,I1503,3,68
356,0.028860,Conduite d'équipement de fabrication de l'ameu...,H2202,3,49
267,0.024672,Métré de la construction,F1108,3,32
285,0.024391,Pose de fermetures menuisées,F1607,3,32
276,0.022816,Montage de structures et de charpentes bois,F1501,3,32
